In [12]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
# Carga de datos.
csv = pd.read_csv('penguins_clean.csv',sep=';')
print(csv.head())

   Culmen length  Culmen depth  Flipper length  Body mass  Especies
0          39.10         18.70          181.00    3750.00       0.0
1          39.50         17.40          186.00    3800.00       0.0
2          40.30         18.00          195.00    3250.00       0.0
3          43.92         17.15          200.92    4201.75       0.0
4          36.70         19.30          193.00    3450.00       0.0


In [6]:
# Convertir dataframe en numpy array
data = np.array(csv)
#todas las filas y todas las columnas
X = data[:,:-1]
#todas filas, solo la última columna
y = data[:,-1]

In [7]:
# Mostrar características de la tabla de datos.
print("Tabla de datos: %d instancias y %d atributos" % (X.shape[0], X.shape[1]))
print("Valores de la clase:", set(y))

Tabla de datos: 344 instancias y 4 atributos
Valores de la clase: {0.0, 1.0, 2.0}


In [8]:
# Test: hold-out split 80-20%. PARTICIÓN EXTERNA
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=42)

valores_test, ocur_test = np.unique(y_test, return_counts=True)
print('Test: ', 'clases:', valores_test, ' ocurrencias: ', ocur_test)

valores_train, ocur_train = np.unique(y_train, return_counts=True)
print('Entrenamiento: ', ' clases:', valores_train, '  ocurrencias:', ocur_train)

Test:  clases: [0. 1. 2.]  ocurrencias:  [32 24 13]
Entrenamiento:   clases: [0. 1. 2.]   ocurrencias: [120 100  55]


In [13]:
# Estandarizar las características de entrenamiento y de test
standardizer = StandardScaler()
X_train = standardizer.fit_transform(X_train)
X_test = standardizer.transform(X_test)

In [17]:
# Hacemos el cross-validation interno para seleccionar los mejores hiperparámetros
from sklearn.svm import SVC
#svc = SVC(C=1, gamma='scale', kernel='rbf')
svc = SVC(C=100, gamma='auto', kernel='linear')
#Se le pasa la arquitectura del modelo pero sin entrenar
#la función cross_val_score nos devuelve la exactitud por cada iteración, utilizando test distintos
results = cross_val_score(svc, X_train, y_train, cv = KFold(n_splits=5, shuffle=True, random_state=42))
print("Resultados por bolsa: ", results)
#Cuando tenemos la media cerca de 1 y la desviación tipíca pequeña cerca de 0, podemos decir que nuestro algoritmo 
#puede llegar a predecir bien
print("Accuracy (media +/- desv.): %0.4f +/- %0.4f" % (results.mean(), results.std()))

Resultados por bolsa:  [0.96363636 0.98181818 1.         0.98181818 0.98181818]
Accuracy (media +/- desv.): 0.9818 +/- 0.0115


In [20]:
# Una vez entrenado y validado el modelo para seleccionar los mejores hyperparameters, utilizamos todos los datos de 
# "train" y "val" para entrenar el modelo definitivo

svc =  SVC(C=100, gamma='auto', kernel='linear')
svc.fit(X_train,y_train)

SVC(C=100, gamma='auto', kernel='linear')

In [21]:
# Calcular la accuracy del conjunto de test
test_results = svc.score(X_test,y_test)
print('Exactitud en test: ', test_results*100, '%')

Exactitud en test:  97.10144927536231 %


In [23]:
# Extraer las predicciones, en lugar de directamente la accuracy
y_pred = svc.predict(X_test)
print('Predicciones:     ', y_pred)
print('Etiquetas reales: ', y_test)

Predicciones:      [1. 1. 1. 1. 2. 2. 2. 1. 0. 1. 2. 0. 0. 0. 2. 0. 0. 1. 2. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 2. 1. 0. 1. 2. 2. 2. 0. 1. 0. 1. 0.
 0. 1. 0. 1. 2. 0. 0. 1. 0. 1. 1. 1. 1. 0. 2. 0. 2. 1. 2. 0. 0.]
Etiquetas reales:  [1. 1. 1. 1. 2. 2. 2. 1. 0. 1. 2. 0. 0. 0. 2. 0. 0. 1. 2. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 2. 1. 0. 1. 2. 2. 2. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 2. 0. 0. 1. 0. 1. 1. 1. 1. 0. 2. 0. 2. 1. 0. 0. 0.]
